In [4]:
import numpy as np
import pandas as pd
import scipy
from scipy.spatial import distance
from sklearn.metrics import f1_score
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score
from scipy.special import rel_entr

In [24]:
Predicted = pd.read_csv("./Stereoscope/Outputs/output_exp0.csv")
Target=pd.read_csv("./Data/exp0/target.csv")

In [25]:
Predicted

,Unnamed: 0,Astro_AMY,Astro_AMY_CTX,Astro_CTX,Astro_HPC,Astro_HYPO,Astro_THAL_lat,Astro_THAL_med,Astro_WM,Ext_Amy_1,...,Inh_Vip,LowQ_2,Micro,Nb_1,Nb_2,OPC_1,OPC_2,Oligo_1,Oligo_2,Unk_1
0,0,0.006442,0.010028,0.029212,0.043119,0.016113,0.059586,0.042702,0.017589,0.015162,...,0.000012,0.000111,0.001737,0.027223,0.013833,0.009457,2.587031e-03,0.000012,0.000009,0.000306
1,1,0.000795,0.011501,0.000056,0.036707,0.052510,0.050317,0.002608,0.033726,0.000030,...,0.000001,0.006178,0.001066,0.026766,0.063694,0.004101,1.646805e-06,0.000013,0.003928,0.000013
2,2,0.000395,0.022163,0.000747,0.064317,0.020189,0.059781,0.005724,0.048527,0.022635,...,0.000002,0.000005,0.003032,0.031940,0.005035,0.000157,2.893755e-03,0.000005,0.000001,0.037158
3,3,0.000677,0.032726,0.033936,0.015951,0.000069,0.058359,0.023421,0.043798,0.009486,...,0.000003,0.012133,0.003346,0.031107,0.000136,0.009020,8.849246e-05,0.000006,0.028829,0.015768
4,4,0.000054,0.035151,0.000007,0.015545,0.013571,0.000001,0.001501,0.055066,0.102475,...,0.000008,0.000010,0.000003,0.031800,0.013194,0.012546,1.019604e-06,0.000001,0.023775,0.035969
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2495,2495,0.012696,0.011750,0.024071,0.044907,0.034711,0.081168,0.073158,0.010974,0.000008,...,0.000112,0.020596,0.000003,0.025600,0.038987,0.013640,1.641987e-04,0.000063,0.001577,0.036219
2496,2496,0.016936,0.027632,0.014160,0.000024,0.006803,0.076797,0.000177,0.041331,0.075147,...,0.000007,0.000002,0.000072,0.029721,0.015599,0.016113,7.680350e-07,0.000013,0.000003,0.030698
2497,2497,0.028579,0.001984,0.000045,0.024544,0.009857,0.000001,0.000005,0.023096,0.101228,...,0.000030,0.012168,0.003178,0.014741,0.048552,0.042675,1.316555e-03,0.000001,0.005318,0.010967
2498,2498,0.000042,0.000007,0.000885,0.036394,0.006837,0.000007,0.009175,0.017060,0.026118,...,0.000280,0.000950,0.003053,0.013103,0.070480,0.023401,1.008943e-05,0.000001,0.000002,0.000013


In [26]:
Target

,Unnamed: 0,cell_abundances_Inh_3,cell_abundances_Ext_Pir,cell_abundances_Oligo_2,cell_abundances_Astro_HYPO,cell_abundances_Ext_L25,cell_abundances_Astro_THAL_lat,cell_abundances_Ext_L5_1,cell_abundances_Ext_L23,cell_abundances_Micro,...,cell_abundances_Inh_Lamp5,cell_abundances_Inh_Pvalb,cell_abundances_Ext_L5_2,cell_abundances_Inh_6,cell_abundances_Astro_AMY,cell_abundances_Ext_ClauPyr,cell_abundances_Astro_HPC,cell_abundances_Astro_WM,cell_abundances_Nb_2,cell_abundances_Inh_5
0,exper0_location_0,0.0,0.0,0.000000,0.000000,1.614244,4.666506,0.0,0.0,0.0,...,0.000000,0.000000,0.395183,1.184708,0.277518,0.541852,0.705551,0.598991,0.792234,0.096642
1,exper0_location_1,0.0,0.0,0.000000,0.000000,0.619247,2.499720,0.0,0.0,0.0,...,0.000000,0.000000,0.310623,0.712052,0.182670,0.516597,0.500672,0.576900,1.419219,0.080714
2,exper0_location_2,0.0,0.0,0.000000,0.000000,0.485019,3.072524,0.0,0.0,0.0,...,0.000000,0.000000,0.304613,0.751698,0.235191,0.512067,0.901390,0.431096,1.370087,0.070171
3,exper0_location_3,0.0,0.0,0.000000,0.158936,0.612778,1.343516,0.0,0.0,0.0,...,0.000000,0.000000,0.323838,0.590181,0.424364,0.695726,0.483868,1.058093,0.637535,0.138405
4,exper0_location_4,0.0,0.0,0.053751,0.234708,0.000000,0.000000,0.0,0.0,0.0,...,0.129119,0.000000,0.307762,1.059323,0.135765,0.249786,0.358404,0.600835,0.297008,0.102129
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2495,exper0_location_2495,0.0,0.0,0.000000,0.000000,1.134150,5.290804,0.0,0.0,0.0,...,0.000000,0.000000,0.576609,0.765302,0.593484,0.801350,0.194004,0.361727,1.520542,0.228784
2496,exper0_location_2496,0.0,0.0,0.000000,0.512848,0.704379,2.299331,0.0,0.0,0.0,...,0.000000,0.000000,0.547743,0.437106,0.584729,0.884072,0.115930,0.413363,1.007258,0.480496
2497,exper0_location_2497,0.0,0.0,0.000000,0.482375,0.000000,0.000000,0.0,0.0,0.0,...,0.000000,0.101733,0.505908,0.410015,0.512465,0.665695,0.155482,0.260211,1.591561,0.143681
2498,exper0_location_2498,0.0,0.0,0.000000,0.272300,0.000000,0.000000,0.0,0.0,0.0,...,0.000000,0.139126,0.664717,0.516948,0.495817,1.315689,0.166854,0.126990,1.208538,0.109524


In [27]:
Predicted.drop(columns="Unnamed: 0", inplace=True)
#Predicted.drop(columns="cell_ID", inplace=True)


In [28]:
Target.drop(columns="Unnamed: 0", inplace=True)

In [29]:
for i in range(len(Target)):
    Target.loc[i,:]=Target.loc[i,:]/Target.loc[i,:].sum()

In [30]:
for i in range(len(Predicted)):
    Predicted.loc[i,:]=Predicted.loc[i,:]/Predicted.loc[i,:].sum()
 

In [31]:
columns_p=[]
for i in range(Predicted.shape[1]):
    columns_p.append(Predicted.columns[i])
    

In [32]:
columns_t=[]
for i in range(Target.shape[1]):
    columns_t.append(Target.columns[i])
    

In [33]:
columns_p=sorted(columns_p)
columns_t=sorted(columns_t)

In [34]:
Predicted=Predicted[columns_p]
Target=Target[columns_t]

In [35]:
Target=Target.to_numpy()
Predicted=Predicted.to_numpy()

## Pearson Correlation

In [36]:
num_celltypes=Predicted.shape[1]
d = np.zeros(num_celltypes)
for i in range(0,num_celltypes):
    p=Predicted[:,i]
    t=Target[:,i]
    prs=(np.corrcoef(p,t,rowvar=False))
    d[i]=prs[0,1]

In [37]:
np.mean(np.nan_to_num(d))

0.6303556861063696

In [38]:
d

array([0.69898869, 0.64287006, 0.50765435, 0.35319696, 0.30661229,
       0.89406522, 0.77086116, 0.60186633, 0.72692189, 0.60286897,
       0.79000196, 0.76726441, 0.74927959, 0.70057496, 0.61544923,
       0.60388996, 0.59846155, 0.32077648, 0.21614631, 0.76396438,
       0.48504618, 0.85157576, 0.73227473, 0.44667544, 0.86483288,
       0.68916154, 0.37944431, 0.54608084, 0.36024759, 0.2045518 ,
       0.6465555 , 0.77292607, 0.4002218 , 0.71167004, 0.68854756,
       0.87659084, 0.85562154, 0.87076853, 0.68382401, 0.82776802,
       0.63111575, 0.7102181 , 0.67929761, 0.75830592, 0.59099703,
       0.8691507 , 0.89797462, 0.47831867, 0.14595052])

## Jenson Shannon Divergence

In [39]:
jns=distance.jensenshannon(Predicted,Target,axis=1)
jns=jns[~np.isinf(jns).squeeze()]
np.mean(jns)

0.45733116211476355

## F1 Score, Precision, Recall

In [40]:
thresholds={0.001,0.01,0.1,0.2,0.3}

In [41]:
d = pd.DataFrame(0, index=np.arange(len(thresholds)), columns=["Precision","Recall","F1-score"])
d.index=thresholds
for t in thresholds:
    Predicted_logical=(Predicted>t).astype(int)
    Target_logical=(Target>t).astype(int)
    d.loc[t,"Precision"]=precision_score(Target_logical,Predicted_logical,average="micro")
    d.loc[t,"Recall"]=recall_score(Target_logical,Predicted_logical,average="micro")
    d.loc[t,"F1-score"]=f1_score(Target_logical,Predicted_logical,average="micro")

/opt/homebrew/Caskroom/miniforge/base/envs/ML/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [42]:
d

,Precision,Recall,F1-score
0.100,0.634420,0.179821,0.280217
0.200,0.218750,0.004636,0.009079
0.300,0.000000,0.000000,0.000000
0.010,0.519401,0.794603,0.628183
0.001,0.478577,0.902039,0.625366


In [ ]:
Prediction